In [ ]:
import pandas as pd
import os
import requests
import time
import random
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [28]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# URL base của trang web
base_url = 'https://www.thivien.net/'

# Danh sách các link bạn muốn crawl
list_link = [
    'https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=3&ReplyType=3&Sort=Views&SortOrder=desc&Page=',
    'https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=2&Sort=Views&SortOrder=desc&Page=',
    'https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=57&Sort=Views&SortOrder=desc&Page=',
    'https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=55&Sort=Views&SortOrder=desc&Page='
]

# Giả lập User-Agent giống như một trình duyệt thực tế
from fake_useragent import UserAgent

ua = UserAgent()
headers = {
    'User-Agent': ua.random
}

poem_link = []

# Duyệt qua từng link
for base_link in list_link:
    print(f"Đang crawl từ link: {base_link}")
    
    # Lặp qua các trang từ 1 đến 10
    for page_idx in tqdm(range(1, 11)):
        try:
            # Tạo URL với số trang
            url = base_link + str(page_idx)
            
            # Gửi yêu cầu HTTP GET để lấy nội dung của trang
            response = requests.get(url, headers=headers)
            
            # Kiểm tra nếu trang tồn tại
            if response.status_code == 200:
                # Sử dụng BeautifulSoup để phân tích nội dung HTML
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Tìm tất cả các thẻ a bên trong h4 có class list-item-header
                poem_links = soup.select('div.list-item h4.list-item-header a')

                # Kiểm tra nếu không có bài thơ nào trên trang
                if not poem_links:
                    print(f"Không có bài thơ nào tại trang {page_idx}")
                    break  # Dừng nếu không có bài thơ nào

                # Duyệt qua từng link và thêm tiền tố nếu thiếu
                for link in poem_links:
                    poem_url = base_url + link['href']
                    poem_link.append(poem_url)

            else:
                print(f"Trang {page_idx} không tồn tại.")
                break  # Dừng nếu trang không tồn tại

        except Exception as e:
            # Bỏ qua lỗi và tiếp tục với link tiếp theo
            print(f"Đã có lỗi xảy ra với trang {page_idx}: {e}")
            continue


Đang crawl từ link: https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=3&ReplyType=3&Sort=Views&SortOrder=desc&Page=


100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


Đang crawl từ link: https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=2&Sort=Views&SortOrder=desc&Page=


100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


Đang crawl từ link: https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=57&Sort=Views&SortOrder=desc&Page=


100%|██████████| 10/10 [00:06<00:00,  1.45it/s]


Đang crawl từ link: https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=55&Sort=Views&SortOrder=desc&Page=


 70%|███████   | 7/10 [00:05<00:02,  1.17it/s]

Không có bài thơ nào tại trang 8


In [30]:
# lưu thành file txt
with open('poem_link.txt', 'w') as f:
    for link in poem_link:
        f.write(link + '\n')

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm  # Thêm thư viện tqdm

# Đọc các link từ file
with open('poem_link.txt', 'r', encoding='utf-8') as file:
    links = file.readlines()

poems = []

# Sử dụng tqdm để theo dõi tiến độ
for link in tqdm(links, desc="Đang thu thập thơ", unit="bài thơ"):
    link = link.strip()  # Loại bỏ khoảng trắng
    try:
        # Thêm header User-Agent
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
        }
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Kiểm tra mã trạng thái

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy tên bài thơ (h1)
        title_tag = soup.select_one('header.page-header > h1')
        title = title_tag.get_text(strip=True) if title_tag else 'Không tìm thấy tiêu đề'

        # Lấy nội dung bài thơ (tất cả các thẻ p trong .poem-content)
        content_tags = soup.select('div.poem-content p')
        content = '\n'.join(tag.get_text(separator='\n', strip=True) for tag in content_tags) if content_tags else 'Không tìm thấy nội dung'

        # Định dạng nội dung để có ký tự xuống dòng
        formatted_content = content.replace('\n', '\\n')  # Thay thế \n với ký tự \n

        # Lưu tên và nội dung bài thơ
        poems.append({'title': title, 'content': formatted_content})

    except Exception as e:
        print(f"Đã có lỗi xảy ra với link: {link}. Lỗi: {e}")



Đang thu thập thơ: 100%|██████████| 370/370 [03:41<00:00,  1.67bài thơ/s]


In [22]:
# Chuyển danh sách poems thành DataFrame
df1 = pd.DataFrame(poems)

# Hiển thị DataFrame
df1.head(5)

,title,content
0,Việt Bắc,-\nMình về mình có nhớ ta?\nMười lăm năm ấy th...
1,Tre Việt Nam,"Tre xanh,\nXanh tự bao giờ?\nChuyện ngày xưa....."
2,Cây dừa,"Cây dừa xanh toả nhiều tàu\nDang tay đón gió, ..."
3,Việt Nam quê hương ta,Việt Nam đất nước ta ơi\nMênh mông biển lúa đâ...
4,Ảnh Bác,Nhà em treo ảnh Bác Hồ\nBên trên là một lá cờ ...


In [23]:
df2 = pd.read_csv('poem.csv')
df2

,content,gender
0,Đêm trong đến không ngờ\nSen cũng thơm quá đỗi...,năm chữ
1,Đền nhiều biết lấy chi người trả\nKhéo lại buồ...,song thất lục bát
2,Nói thật đúng từ biển dâu khép mở\nThật rõ ra ...,tám chữ
3,Thăm con ở trại nhi đồng\nMột ngày xuân đẹp nắ...,lục bát
4,Tất cả lý thuyết đều màu xám\nMà cây đời thì m...,sáu chữ
...,...,...
358895,Bên đình trăng rọi thẩm cây xanh\nChậm rãi thờ...,tám chữ
358896,Ai nói miền nam chẳng lạnh lùng\nKhông lạnh sa...,tám chữ
358897,Nàng thơ áo trắng bước kiêu sa\nNgây ngất hồn ...,tám chữ
358898,Đời quá buồn trở thành tác giả\nCuộc đời tôi c...,tám chữ


In [24]:
df2 = df2[df2['gender'] == 'lục bát']

# Hiển thị kết quả
df2

,content,gender
3,Thăm con ở trại nhi đồng\nMột ngày xuân đẹp nắ...,lục bát
6,Hôm nay có phải là thu\nMây năm xưa đã phiêu d...,lục bát
7,Khóc cho vơi bớt nỗi buồn\nKhóc cho quên hết n...,lục bát
10,Tò he xanh đỏ tím vàng\nMẹ mua mỗi độ chợ làng...,lục bát
11,Thương chiều lá rụng bên song\nQuạnh hiu một b...,lục bát
...,...,...
309069,Trời xui nên buổi đi săn\nTrạng tới nơi này th...,lục bát
309070,Miền trung khúc ruột nát nhàu\nCả nước rơi lệ ...,lục bát
309071,Nào ngờ mưa lũ dâng tràn\nNhà nhà ngập nước cơ...,lục bát
309072,Tháng mười lục bát nở hoa\nĐường thi kết trái ...,lục bát


In [25]:
# Thêm cột 'title' cho df2 và gán giá trị None cho các hàng
df2['title'] = None

# Loại bỏ cột 'gender' của df2
df2 = df2[['title', 'content']]

# Gộp df1 và df2
df_full = pd.concat([df1, df2], ignore_index=True)

# Loại bỏ các dòng có "Không tìm thấy tiêu đề" hoặc "Không tìm thấy nội dung"
df_full = df_full[~df_full['title'].isin(['Không tìm thấy tiêu đề'])]
df_full = df_full[~df_full['content'].isin(['Không tìm thấy nội dung'])]


# lưu file 
df_full.to_csv('data_no_clean.csv', index=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18356\2202503238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['title'] = None


In [26]:
df_full

,title,content
0,Việt Bắc,-\nMình về mình có nhớ ta?\nMười lăm năm ấy th...
1,Tre Việt Nam,"Tre xanh,\nXanh tự bao giờ?\nChuyện ngày xưa....."
2,Cây dừa,"Cây dừa xanh toả nhiều tàu\nDang tay đón gió, ..."
3,Việt Nam quê hương ta,Việt Nam đất nước ta ơi\nMênh mông biển lúa đâ...
4,Ảnh Bác,Nhà em treo ảnh Bác Hồ\nBên trên là một lá cờ ...
...,...,...
179310,None,Trời xui nên buổi đi săn\nTrạng tới nơi này th...
179311,None,Miền trung khúc ruột nát nhàu\nCả nước rơi lệ ...
179312,None,Nào ngờ mưa lũ dâng tràn\nNhà nhà ngập nước cơ...
179313,None,Tháng mười lục bát nở hoa\nĐường thi kết trái ...
